In [46]:
import keras
from keras import layers
from keras import models
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization as BatchNorm, Activation, Conv2D, Add, GlobalMaxPool2D, Flatten, Softmax, Cropping1D, Lambda, Maximum
import numpy as np
import os
from itertools import islice
from matplotlib import pyplot as plt
import multiprocessing as mp
tf.config.optimizer.set_jit(True)
print('done with importing')

done with importing


In [47]:
def res_unit1(inp, a, b, block_id, layer_id, activation="elu",):
    Nonlinearity = lambda name: Activation(activation=activation, name=name)

    bn1 = BatchNorm(name=f"{block_id}/{layer_id}/bn1")(inp)
    elu1 = Nonlinearity(name=f"{block_id}/{layer_id}/{activation}1")(bn1)
    conv1 = Conv2D(filters=a, kernel_size=(1,1), name=f"{block_id}/{layer_id}/conv1")(elu1)

    bn2 = BatchNorm(name=f"{block_id}/{layer_id}/bn2")(conv1)
    elu2 = Nonlinearity(name=f"{block_id}/{layer_id}/{activation}2")(bn2)
    conv2 = Conv2D(filters=a, kernel_size=(3,3), padding="same", name=f"{block_id}/{layer_id}/conv2")(elu2)

    bn3 = BatchNorm(name=f"{block_id}/{layer_id}/bn3")(conv2)
    elu3 = Nonlinearity(name=f"{block_id}/{layer_id}/{activation}3")(bn3)
    conv3 = Conv2D(filters=b, kernel_size=(1,1), name=f"{block_id}/{layer_id}/conv3")(elu3)

    return Add(name=f"{block_id}/{layer_id}/add")([inp, conv3])

def res_unit2(inp, a, b, block_id, layer_id, activation="elu"):
    Nonlinearity = lambda name: Activation(activation=activation, name=name)

    bn = BatchNorm(name=f"{block_id}/{layer_id}/bn0")(inp)
    elu = Nonlinearity(name=f"{block_id}/{layer_id}/{activation}0")(bn)

    shortcut = Conv2D(filters=b, kernel_size=(1,1), strides=2, name=f"{block_id}/{layer_id}/shortcut-conv")(elu)

    conv0 = Conv2D(filters=a, kernel_size=(1,1), strides=2, name=f"{block_id}/{layer_id}/conv0")(elu)

    bn1 = BatchNorm(name=f"{block_id}/{layer_id}/bn1")(conv0)
    elu1 = Nonlinearity(name=f"{block_id}/{layer_id}/{activation}1")(bn1)
    conv1 = Conv2D(filters=a, kernel_size=(3,3), padding="same", name=f"{block_id}/{layer_id}/conv1")(elu1)

    bn2 = BatchNorm(name=f"{block_id}/{layer_id}/bn2")(conv1)
    elu2 = Nonlinearity(name=f"{block_id}/{layer_id}/{activation}2")(bn2)
    conv2 = Conv2D(filters=b, kernel_size=(1,1), name=f"{block_id}/{layer_id}/conv2")(elu2)

    return Add(name=f"{block_id}/{layer_id}/add")([shortcut, conv2])

In [98]:
bs = 128

def create_model(activation="elu"):
    img = Input(shape=(28,28,1), name="img")
    conv1 = Conv2D(filters=32, kernel_size=(7,7), name="0/0/conv")(img)
    elu1 = Activation(activation=activation, name=f"0/0/{activation}")(conv1)
    bn1 = BatchNorm(name="0/0/bn")(elu1)

    layer1_1 = res_unit1(bn1, 16, 32, 1, 1, activation=activation)
    layer1_2 = res_unit1(layer1_1, 16, 32, 1, 2, activation=activation)
    layer1_3 = res_unit1(layer1_2, 16, 32, 1, 3, activation=activation)

    layer2_1 = res_unit2(layer1_3, 32, 64, 2, 1, activation=activation)
    layer2_2 = res_unit1(layer2_1, 32, 64, 2, 2, activation=activation)
    layer2_3 = res_unit1(layer2_2, 32, 64, 2, 3, activation=activation)

    layer3_1 = res_unit2(layer2_3, 64, 128, 3, 1, activation=activation)
    layer3_2 = res_unit1(layer3_1, 64, 128, 3, 2, activation=activation)
    layer3_3 = res_unit1(layer3_2, 64, 128, 3, 3, activation=activation)

    layer4_1 = res_unit2(layer3_3, 128, 256, 4, 1, activation=activation)
    layer4_2 = res_unit1(layer4_1, 128, 256, 4, 2, activation=activation)
    layer4_3 = res_unit1(layer4_2, 128, 256, 4, 3, activation=activation)

    layer5_1 = res_unit2(layer4_3, 256, 512, 5, 1, activation=activation)
    layer5_2 = res_unit1(layer5_1, 256, 512, 5, 2, activation=activation)
    layer5_3 = res_unit1(layer5_2, 256, 512, 5, 3, activation=activation)

    average = GlobalMaxPool2D()(layer5_3)
    flatten = Flatten()(average)
    out = Dense(10, activation="sigmoid")(flatten) #changed to 10
    
    return Model(inputs=[img], outputs=[out])

In [ ]:
from sklearn.datasets import fetch_openml
X, Y = fetch_openml(name='mnist_784', return_X_y=True, cache=False)


# Rescale grayscale values to [0,1].
X = X / 255.0

# Shuffle the input: create a random permutation of the integers between 0 and the number of data points and apply this
# permutation to X and Y.
# NOTE: Each time you run this cell, you'll re-shuffle the data, resulting in a different ordering.
shuffle = np.random.permutation(np.arange(X.shape[0]))
X, Y = X[shuffle], Y[shuffle]

print('data shape: ', X.shape)
print('label shape:', Y.shape)

# Set some variables to hold test, dev, and training data.
test_data, test_labels = X[61000:], Y[61000:]
train_data, train_labels = X[:60000], Y[:60000]
mini_train_data, mini_train_labels = X[:1000], Y[:1000]

In [ ]:
from keras.utils import to_categorical
y_binary = to_categorical(train_labels.reshape(60000,1))

In [100]:
model = create_model("relu")
model.compile(
      loss='categorical_crossentropy',
      optimizer=tf.keras.optimizers.Adam(),
      metrics=[metrics.AUC(), metrics.Precision(), metrics.Recall()]
  )

In [101]:
history = model.fit(x=train_data.reshape(60000,28,28,1), y=y_binary, batch_size=1, epochs=1)

ValueError: A target array with shape (60000, 10) was passed for an output of shape (None, 1) while using as loss `categorical_crossentropy`. This loss expects targets to have the same shape as the output.

In [90]:
shaper = test_data.shape[0]
y_pred = model.predict(test_data.reshape(shaper, 28,28,1))
print(max(y_pred)


[[1.0000000e+00 9.9608243e-01 7.1525574e-07 ... 9.9999982e-01
  8.3446503e-07 2.3245811e-06]
 [1.0000000e+00 9.0110898e-02 1.1920929e-07 ... 1.0000000e+00
  1.4901161e-07 4.5299530e-06]
 [1.0000000e+00 9.9671400e-01 2.6524067e-06 ... 9.9999982e-01
  9.2387199e-07 8.8721514e-05]
 ...
 [9.9999988e-01 8.5439628e-01 7.7486038e-07 ... 9.9999976e-01
  2.5331974e-06 7.2658062e-05]
 [1.0000000e+00 7.5950909e-01 0.0000000e+00 ... 1.0000000e+00
  8.9406967e-08 2.3275614e-05]
 [1.0000000e+00 8.9550841e-01 8.9406967e-08 ... 9.9999994e-01
  1.1920929e-07 6.6369772e-05]]


In [93]:
y_try = []
for val in y_pred:
    y_try.append(np.argmax(val))
y_try

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 7,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 7,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 7,
 0,
 7,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 7,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
